# Кросс-валидация ARIMA (пакет [`sktime`](https://www.sktime.net/en/stable/))

In [ ]:
import numpy as np
import pandas as pd

from sktime.forecasting.arima import ARIMA, AutoARIMA
from sktime.utils.plotting import plot_series
from sktime.split import temporal_train_test_split, ExpandingWindowSplitter, SlidingWindowSplitter, SingleWindowSplitter
from sktime.performance_metrics.forecasting import MeanSquaredError, MeanAbsoluteError
from sktime.forecasting.model_evaluation import evaluate

import pandas_datareader.data as web

# настройки визуализация
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)
# Не показывать ValueWarning, ConvergenceWarning из statsmodels
from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning
warnings.simplefilter('ignore', category=ValueWarning)
warnings.simplefilter('ignore', category=ConvergenceWarning)

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) месячные данные по Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity (Symbol [`GS10`](https://fred.stlouisfed.org/series/GS10)) с 2000-01-01 по 2023-12-31 и создадим датафрейм `y`

In [ ]:
y = web.DataReader(name='GS10', data_source='fred', start='2000-01', end='2023-12')
y.index = pd.period_range(start='2000-01', end='2023-12', freq='M')
# длина ряда
len(y)

## Валидация через разделение на обучающую и тестовую выборки

Используем метод [`temporal_train_test_split`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.split.temporal_train_test_split.html) из пакета `sktime`

В качестве метрики выберем [`MeanSquaredError`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.performance_metrics.forecasting.MeanSquaredError.html)

Разберём на примере ARIMA(2,1,2)

In [ ]:
# специфицируем модель
forecaster = ARIMA(order=(2,1,2), trend='n')

# разбиваем выбору на обучающую (первые 80%) и тестовую
y_train, y_test = temporal_train_test_split(y, train_size=0.8)

# Обучаем модель на тестовом множестве
forecaster.fit(y_train)

# прогнозируем для тестовых наблюдений
y_pred = forecaster.predict(fh = y_test.index)

# инициализируем метрику
metric = MeanSquaredError(square_root=False)
# вычисляем метрику на данных
metric.evaluate(y_test, y_pred)

In [ ]:
plot_series(y_train, y_test, y_pred, labels=['train', 'test', 'pred'])

plt.show()

Используем  [`SingleWindowSplitter`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.split.SingleWindowSplitter.html) 

Для вычисления метрики для каждого фолда используем метод [`MeanSquaredError`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.performance_metrics.forecasting.MeanSquaredError.html)

Для проведения кросс-валидации используем [`evaluate`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.model_evaluation.evaluate.html)

Разберём на примере ARMA(2,1,2)

In [ ]:
# специфицируем модель
forecaster = ARIMA(order=(2,1,2), trend='n')

# разбиваем выбору на обучающую (первые 150) и тестовую
cv = SingleWindowSplitter(fh=np.arange(1, len(y)-150+1), window_length=150)


# инициализируем метрику
metric = [MeanSquaredError(square_root=False), MeanAbsoluteError()]

df = evaluate(forecaster=forecaster, y=y, cv=cv, strategy="refit", return_data=False, scoring=metric)
df

In [ ]:
# MSE
df.iloc[:,[0,1]]

## Валидация методом k-Fold (расширяем обучающую выборку)

Используем  [`ExpandingWindowSplitter`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.split.ExpandingWindowSplitter.html) 

Для вычисления метрики для каждого фолда используем метод [`MeanSquaredError`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.performance_metrics.forecasting.MeanSquaredError.html)

Для проведения кросс-валидации используем [`evaluate`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.model_evaluation.evaluate.html)

Разберём на примере ARMA(2,3)

In [ ]:
# специфицируем модель
forecaster = ARIMA(order=(2,1,2), trend='n')

# разбиваем выбору на обучающую (первые 100) и тестовую
cv = ExpandingWindowSplitter(fh=np.arange(1, 11), initial_window=100, step_length=10)

# инициализируем метрику
metric = [MeanSquaredError(square_root=False), MeanAbsoluteError()]

df = evaluate(forecaster=forecaster, y=y, cv=cv, strategy="refit", return_data=False, scoring=metric)
df

In [ ]:
# средняя MSE
df.iloc[:,[0,1]].mean()

## Валидация методом k-Fold (скользящая обучающая выборка)

Используем метод [`SlidingWindowSplitter`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.split.SlidingWindowSplitter.html)

Для вычисления метрики для каждого фолда используем метод [`MeanSquaredError`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.performance_metrics.forecasting.MeanSquaredError.html)

Для проведения кросс-валидации используем [`evaluate`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.model_evaluation.evaluate.html)

Разберём на примере ARMA(2,3)

In [ ]:
# специфицируем модель
forecaster = ARIMA(order=(2,1,2), trend='n')

# разбиваем выбору на обучающую (первые 100) и тестовую
cv = SlidingWindowSplitter(fh=np.arange(1, 11), initial_window=100, step_length=10)

# инициализируем метрику
metric = [MeanSquaredError(square_root=False), MeanAbsoluteError()]

df = evaluate(forecaster=forecaster, y=y, cv=cv, strategy="refit", return_data=False, scoring=metric)
df

In [ ]:
# средняя MSE
df.iloc[:,[0,1]].mean()